In [1]:
from result_saver import SaverProvider

provider = SaverProvider()

In [2]:
backend = provider.get_backend('ibm_kyoto')

In [4]:
from qiskit_ibm_provider import IBMProvider

providerIBM = IBMProvider()

In [35]:
jobs = provider.jobs(backend_name='ibm_kyoto', limit=500)

In [96]:
import re

def parse_string_RepCodes(tags):
    string_ = tags[0]
    pattern = re.compile(r"RESET (\d+), (\d+), (\d+), xbasis=(\w+)")

    match = pattern.search(string_)
    if match:
        return {
            'distance': int(match.group(1)),
            'rounds': int(match.group(2)),
            'logical': int(match.group(3)),
            'xbasis': True if (match.group(4)) == 'True' else False
        }
    else:
        return None
        

def is_calib_str(tags):

    if 'Calibration' not in tags[0]:
        return False, False

    else:
        double = True if len(tags) > 1 else False

        return True, double


print(parse_string_RepCodes(["120 del w/ RESET 51, 50, 0, xbasis=False"]))
is_calib_str(["Calibration, shots 31496"])

{'distance': 51, 'rounds': 50, 'logical': 0, 'xbasis': False}


(True, False)

In [105]:
from tqdm import tqdm

entries = []
calib_count = 0
for job in tqdm(jobs):
    if str(job.status()) != 'JobStatus.DONE':
        continue
    tags = job.tags()

    calib, double = is_calib_str(tags)

    if not calib:
        parsed = parse_string_RepCodes(tags)
        additional_metadata = {
            'descr': 'With Resets 50 rounds',
            'code': "RepetitionCodeCircuit",
            'distance': parsed['distance'],
            'rounds': parsed['rounds'],
            'logical': parsed['logical'],
            'xbasis': parsed['xbasis'],
            'resets': True,
            'job_status': "JobStatus.RUNNING",
        }
    else:
        if calib_count % 4 < 2:
            sampled_state = '0'*127
        else:
            sampled_state = '1'*127
        calib_count += 1
        additional_metadata = {
            'num_qubits': 127,
            "sampled_state": sampled_state,
            'optimization_level': 0,
            'double_msmt': double,
            'job_status': "JobStatus.RUNNING",
        }
    
    entry_dict = {
        'creation_date': str(job.creation_date()),
        'notebook_name': 'Kyoto_retrieve_jobs.ipynb',
        'bakend_name': 'ibm_kyoto',
        'backend_options': job.backend_options(),
        'additional_metadata': additional_metadata,
        'job_id': job.job_id(),
        'tags': tags,
    }
    entries.append(entry_dict)


100%|██████████| 473/473 [00:00<00:00, 126872.53it/s]


In [106]:
len(entries)

100

In [107]:
import json

file_path = '/Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/.Scratch/job_metadata.json'
with open(file_path, 'r') as file:
    metadata_list = json.load(file)

for entry in entries:
    metadata_list.append(entry)

with open(file_path, 'w') as file:
    json.dump(metadata_list, file, indent=4)